In [1]:
import torch
import numpy as np
import pickle as pkl
from glob import glob
import os
import argparse
import regex as re
import pandas as pd
import statistics
import copy

from MisInfoSpread import MisInfoSpread
from MisInfoSpread import MisInfoSpreadState

def flatten(state):
    return [val * i for val, adj in zip(state.node_states, state.adjacency_matrix) for i in adj]

In [2]:
def run_inference( dataset_path, model_path, nodes, max_steps, st, count_inf, count_actions):
    
    # device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
    print(f"Using device: {device}")

    misinfo = MisInfoSpread(num_nodes=nodes, max_time_steps=max_steps, 
                        trust_on_source=st, count_infected_nodes=count_inf, 
                        count_actions=count_actions)

    model = misinfo.get_nnet_model().to(device)
    model.load_state_dict(torch.load(model_path , map_location=torch.device(device)))

    states = pkl.load(open(dataset_path, 'rb'))
    # for state in states:
    #     state.node_states = [int(x) for x in state.node_states]
    #     for i in range(len(state.adjacency_matrix)):
    #         for j in range(len(state.adjacency_matrix[i])):
    #             if state.adjacency_matrix[i][j] != 0:
    #                 state.adjacency_matrix[i][j] = 1
    #             else:
    #                 state.adjacency_matrix[i][j] = 0

    candidate_nodes = misinfo.find_neighbor_batch(states)

    actions_dict = {i: {'actions': [], 'infRate': 0} for i in range(len(states))}

    while any(candidate_node for candidate_node in candidate_nodes):
        blockernode_np = []
        count = 0
        for state, cand_nodes in zip(states, candidate_nodes):
            print("Processing states ", count, end='\r')
            if cand_nodes:
                expectation_values = []
                for cand_node in cand_nodes:
                    temp_ns, _, _ = misinfo.step(copy.deepcopy(state), [cand_node])
                    output_tensor = torch.FloatTensor(flatten(temp_ns)).view(1, -1).to(device)
                    expected_infection = model(output_tensor).detach().cpu().numpy()
                    expectation_values.append( (expected_infection, cand_node) )

                # sort the expectation values based on the expected infection
                expectation_values.sort(key=lambda x: x[0], reverse=True)

                if len(expectation_values) < count_actions:
                    blockernode_np.append([node for _, node in expectation_values])
                    actions_dict[count]['actions'].append([node for _, node in expectation_values])
                else:
                    blockernode_np.append([node for _, node in expectation_values[:count_actions]])
                    actions_dict[count]['actions'].append([node for _, node in expectation_values[:count_actions]])
            else:
                blockernode_np.append([])

            count += 1
        next_states, rewards, done = misinfo.step_batch(states, blockernode_np)
        states = next_states
        candidate_nodes = misinfo.find_neighbor_batch(states)
        # print count of dones
        print("Done: ", done.count(True), " "*20)
        if all(done):
            break
    
    inf_rate = []
    count = 0
    for state in states:
        inf_rate.append(state.node_states.count(-1.0)/len(state.node_states))
        actions_dict[count]['infRate'] = state.node_states.count(-1.0)/len(state.node_states)
        count += 1

    mean = round(statistics.mean(inf_rate), 4)
    std_dev = round(statistics.stdev(inf_rate), 4)
    print(f"Mean: {mean}, Std Dev: {std_dev}")
    return mean, std_dev, actions_dict

In [3]:
model = "saved_models/target_model_1_1_mn10_ms100_st1.0.pt"
dataset_path = "/Users/bittu/Desktop/InfoSpread-server/dataset/generate_dataset/deg_dataset/10/10_deg_3.pkl"

mean, std_dev, actions_dict = run_inference(dataset_path, model, 10, 50, 1.0, 1, 1)


Using device: mps
Done:  0                     
Done:  25                     
Done:  984                     
Done: sing states  999 997                     
Done:  1000                     
Mean: 0.2096, Std Dev: 0.0872


In [7]:
for key, item in actions_dict.items():
    if len(item['actions']) < 3 and len(item['actions']) > 1:
        print(key, item['actions'], item['infRate'])

14 [9, 7] 0.3
31 [6, 5] 0.3
37 [6, 8] 0.4
61 [9, 4] 0.3
113 [5, 9] 0.4
120 [3, 6] 0.4
176 [4, 6] 0.3
179 [8, 2] 0.2
231 [9, 7] 0.2
233 [9, 5] 0.3
242 [7, 0] 0.2
354 [1, 3] 0.3
400 [9, 0] 0.3
427 [4, 9] 0.3
476 [5, 0] 0.2
505 [9, 7] 0.3
528 [8, 5] 0.4
606 [4, 7] 0.4
612 [1, 6] 0.2
688 [0, 1] 0.4
805 [6, 1] 0.2
822 [3, 0] 0.3
871 [5, 8] 0.2
902 [3, 0] 0.4
937 [5, 7] 0.4


In [4]:
actions_dict

{0: {'actions': [[3], [0], [7]], 'infRate': 0.2},
 1: {'actions': [[5], [9], [7]], 'infRate': 0.2},
 2: {'actions': [[0], [5], [3]], 'infRate': 0.1},
 3: {'actions': [[0], [6], [7]], 'infRate': 0.3},
 4: {'actions': [[9], [6], [4]], 'infRate': 0.1},
 5: {'actions': [[3], [4], [8]], 'infRate': 0.2},
 6: {'actions': [[9], [1], [7]], 'infRate': 0.2},
 7: {'actions': [[4], [9], [1]], 'infRate': 0.3},
 8: {'actions': [[4], [6], [5]], 'infRate': 0.2},
 9: {'actions': [[6], [8], [1]], 'infRate': 0.2},
 10: {'actions': [[4], [7], [9]], 'infRate': 0.2},
 11: {'actions': [[4], [3], [6]], 'infRate': 0.2},
 12: {'actions': [[5], [4], [1]], 'infRate': 0.2},
 13: {'actions': [[9], [1], [6]], 'infRate': 0.1},
 14: {'actions': [[9], [7]], 'infRate': 0.3},
 15: {'actions': [[9], [1], [4]], 'infRate': 0.3},
 16: {'actions': [[1], [8], [4]], 'infRate': 0.2},
 17: {'actions': [[4], [3], [8]], 'infRate': 0.1},
 18: {'actions': [[2], [7], [4]], 'infRate': 0.2},
 19: {'actions': [[1], [4], [2]], 'infRate': 0

In [5]:
for key, item in actions_dict.items():
    actions_dict[key]['actions'] = [action_item[0] for action_item in item['actions']]

In [6]:
import json

# save actions dict as json file with intend

with open("actions_r3.json", "w") as outfile: 
    json.dump(actions_dict, outfile, indent=4)